In [ ]:
# Import required modules
import torch
import torch.optim as optim
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
import torchvision.models as models

from google.colab import drive

import seaborn as sn
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, classification_report, roc_curve, auc

In [ ]:
# Mount drive and retrieve path
drive.mount('/content/drive')

train_path = '/content/drive/MyDrive/Parkinsons-DaTscan-master/Implementation/TRAIN_CROP/'
test_path = '/content/drive/MyDrive/Parkinsons-DaTscan-master/Implementation/TEST_CROP/'
val_path = '/content/drive/MyDrive/Parkinsons-DaTscan-master/Implementation/VAL_CROP/'

In [ ]:
# Define preprocessing transforms
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])
])

In [ ]:
# Create train and test data loaders
batch_size = 30
num_epochs = 30

train_loader = torch.utils.data.DataLoader(ImageFolder(train_path, transform=transform_train), batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(ImageFolder(test_path, transform=transform_test), batch_size=batch_size, shuffle=True)
val_loader = torch.utils.data.DataLoader(ImageFolder(val_path, transform=transform_test), batch_size=batch_size, shuffle=True)

In [ ]:
# Load pre-trained vgg16 model, Freeze pre-trained layers and Modify last layer
vgg19 = models.vgg19(pretrained=True)

for param in vgg19.parameters():
    param.requires_grad = False

num_classes = 2
vgg19.classifier[-1] = torch.nn.Linear(4096, num_classes)

In [ ]:
# Define loss function and optimizer
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(vgg19.parameters(), lr=0.001)

accuracy_list = []
loss_list = []

# Train the model
for epoch in range(num_epochs):
    running_loss = 0.0
    total_correct = 0
    total_samples = 0
    for i, (images, labels) in enumerate(train_loader, 0):
        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward + backward + optimize
        outputs = vgg19(images)
        _, predicted = torch.max(outputs.data, 1)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # Update metrics utilities
        total_samples += labels.size(0)
        total_correct += (predicted == labels).sum().item()

        # Print statistics
        running_loss += loss.item()
        if i % 10 == 9:
            print('[Epoch %d, Batch %d] Loss: %.3f' %(epoch + 1, batch_size, running_loss / 10))
            running_loss = 0.0

    accuracy = 100 * total_correct / total_samples
    accuracy_list.append(accuracy)

    epoch_loss = running_loss / len(train_loader)
    loss_list.append(epoch_loss)

In [ ]:
# Plot the accuracy versus epoch
plt.plot(range(1, num_epochs + 1), accuracy_list)
plt.xlabel('Epoch')
plt.ylabel('Accuracy (%)')
plt.title('Accuracy vs. Epoch')
plt.show()

In [ ]:
# Plot the loss versus epoch
plt.plot(range(1, num_epochs + 1), loss_list)
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Loss vs. Epoch')
plt.show()

In [ ]:
# Evaluate on validation set
correct = 0
total = 0
with torch.no_grad():
    for data in val_loader:
        images, labels = data
        outputs = vgg19(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy on validation set: %d %%' % (100 * correct / total))

In [ ]:
# Evaluate on test set
correct = 0
total = 0
actual = []
prediction = []

with torch.no_grad():
    for data in test_loader:
        images, labels = data
        outputs = vgg19(images)
        _, predicted = torch.max(outputs.data, 1)
        actual.extend(labels.tolist())
        prediction.extend(predicted.tolist())
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy on test set: %d %%' % (100 * correct / total))

In [ ]:
# Create confusion matrix on test set
confusion_mtx = confusion_matrix(actual, prediction)
ax = plt.axes()
sn.heatmap(confusion_mtx, annot=True,annot_kws={"size": 25}, cmap="Blues", ax = ax)
ax.set_title('Test Accuracy', size=14)
plt.show()

In [ ]:
# Calculate scores
TN, FP, FN, TP = confusion_mtx.ravel()

precision = TP / (TP + FP)
recall = TP / (TP + FN)
specificity = TN / (TN + FP)
f1 = 2 * (precision * recall) / (precision + recall)

print("Precision:", precision)
print("Recall (Sensitivity):", recall)
print("Specificity:", specificity)
print("F1 Score:", f1)

In [ ]:
print("Classification Report:")
report = classification_report(actual, prediction, target_names=["Yes", "No"])
print(report)

In [ ]:
# Calculate ROC curve and AUC
fpr, tpr, _ = roc_curve(actual, prediction)
roc_auc = auc(fpr, tpr)

# Plot ROC curve
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (AUC = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC)')
plt.legend(loc="lower right")
plt.show()